In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pyarrow
import os
os.system(f"pip install unsloth fastapi uvicorn pyngrok peft transformers==4.56.2 pyarrow=={pyarrow.__version__}")

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

from peft import PeftModel
model = PeftModel.from_pretrained(model, "hg_username/your_model_name")
FastLanguageModel.for_inference(model)

In [ ]:
import asyncio
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

lock = asyncio.Lock()
waiting_count = 0
MAX = 10

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

class RequestBody(BaseModel):
    data: str

@app.post("/evaluate")
async def evaluate(body: RequestBody):
    global waiting_count

    if waiting_count >= MAX:
        return {"status": "unavailable", "message": "Server is busy, please try again later."}

    waiting_count += 1
    try:
        async with lock:
            prompt = alpaca_prompt.format(
                "Evaluate the candidate's fit for the job and respond with a JSON object containing verdict, strengths, gaps, and explanation.",
                f"{body.data}",
                ""
            )
            inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
            outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
            decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            response_text = decoded.split("### Response:")[-1].strip()
            return {"status": "ok", "result": response_text}
    finally:
        waiting_count -= 1

In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok

ngrok.set_auth_token("ngrok_token")
nest_asyncio.apply()

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()